In [1]:
import pandas as pd
import numpy as np
import os
import openai
import dotenv
from openai import OpenAI
import sys
import pandas as pd
import json
from dotenv import load_dotenv
import re

In [ ]:
# Primer carreguem els csv

df = pd.read_csv('..//datasets//final_dataset_v1.csv', low_memory=False)
df_2024 = pd.read_csv('..//datasets//2024.csv', low_memory=False)
df_id = pd.read_csv('..//datasets//data_by_id_desembre_2024.csv', low_memory=False)

In [ ]:
id_set = set(df_id['id'])

In [81]:
df.columns

Index(['id', 'data', 'entradeta', 'permatitle', 'avantitol', 'tematiques',
       'tags', 'titol', 'titol_portada', 'comarques', 'cos', 'redactor',
       'imatges'],
      dtype='object')

In [82]:
df_id.columns

Index(['id', 'cos', 'redactor'], dtype='object')

In [83]:
df_2024.columns

Index(['data', 'entradeta', 'permatitle', 'avantitol', 'titol_portada',
       'tematiques', 'id', 'titol', 'imatges', 'comarques', 'tags'],
      dtype='object')

In [ ]:
# Prenem les notícies de desembre de 2024
df_2024 = df_2024[df_2024['id'].isin(id_set)]

# eliminem les columnes que no ens fan falta: entradeta, permatitle, tematiques, fonts
df_id = df_id.drop(columns=['entradeta', 'permatitle', 'tematiques', 'fonts', 'mediadestacats', 'tts_audios', 'data_publicacio'])
df_id = df_id.drop(columns=['especials', 'programes_radio', 'canals_directes_embedats', 'lloc', 'programes_radio', 'audios', 'programes_tv'])
df_id = df_id.drop(columns=['autors', 'videos', 'comarques', 'tags', 'noticies', 'canals', 'agrupacions'])
df_id = df_id.drop(columns=['avantitol', 'state', 'imatges', 'domini', 'titol', 'data_modificacio', 'titol_portada'])
df_2024 = df_2024.drop(columns=['tipologia', 'data_modificacio', 'data_publicacio', 'domini', 'agrupacions', 'autors'])
df_2024 = df_2024.drop(columns=['canals', 'programes_tv', 'programes_radio', 'especials', 'canals_directes_embedats', 'tematiques_old', 'lloc'])

# Canviem el nom de la columna "date" de df_2024 per "data"
df_2024 = df_2024.rename(columns={'date': 'data'})

In [ ]:
# Fem un merge de les notícies de desembre de 2024
df_merged = pd.merge(df_2024, df_id, on='id', how='inner')

            data                                          entradeta  \
0     01/12/2024  Hi ha almenys 6 passatgers ferits greus, quatr...   
1     01/12/2024  El Hajduk Split referma el liderat a la Lliga ...   
2     01/12/2024  Els plans de creixement de Dorna passen per ex...   
3     01/12/2024  El sacrifici de l'equip de Dani Miret redueix ...   
4     01/12/2024  Una nova llei, inèdita al món, reconeix que te...   
...          ...                                                ...   
2388  31/12/2024  Hem resumit l'any amb una cinquantena de fotog...   
2389  31/12/2024  L'índex de referència baixa per novena vegada ...   
2390  31/12/2024       El 2024 acabarà amb domini del temps estable   
2391  31/12/2024                                                NaN   
2392  31/12/2024                                                NaN   

                                             permatitle avantitol  \
0     Almenys 2 morts en l'accident d'un autocar de ...       NaN   
1     Iva

In [ ]:
# Reordenem les columnes
nou_ordre = ['id', 'data', 'entradeta', 'permatitle', 'avantitol', 'tematiques',
               'tags', 'titol', 'titol_portada', 'comarques', 'cos', 'redactor', 'imatges']

# Reordenar el DataFrame
df_merged = df_merged[nou_ordre]

,id,data,entradeta,permatitle,avantitol,tematiques,tags,titol,titol_portada,comarques,cos,redactor,imatges
0,3324751,01/12/2024,"Hi ha almenys 6 passatgers ferits greus, quatr...",Almenys 2 morts en l'accident d'un autocar de ...,NaN,"[{'id': 'PUACT_SOCIET', 'desc': 'Societat', 'm...",NaN,Almenys dos morts en l'accident d'un autocar d...,NaN,NaN,<!--RichText--><p>Almenys dues persones han mo...,Redacció,[{'text': 'https://img.3cat.cat/multimedia/png...
1,3324758,01/12/2024,El Hajduk Split referma el liderat a la Lliga ...,Ivan Rakitic reviu les nits felices sortint gl...,futbol,"[{'id': 'PUE3_FUTINT', 'desc': 'Futbol Interna...",NaN,Ivan Rakitic reviu les nits felices sortint gl...,Rakitic reviu les nits felices sortint gloriós...,NaN,<!--RichText--><p>Després d'anys al cim del fu...,Redacció Esport3,[{'text': 'https://img.3cat.cat/multimedia/png...
2,3324755,01/12/2024,Els plans de creixement de Dorna passen per ex...,Al congelador l'opció que Madrid aculli un Gra...,motogp,"[{'id': 'PUE3_MOTOGP', 'desc': 'Motor - Moto G...",NaN,Al congelador l'opció que Madrid aculli un gra...,Al congelador l'opció que Madrid aculli un GP ...,NaN,<!--RichText--><p>Com si es tractés d'un efect...,Redacció Esport3,[{'text': 'https://img.3cat.cat/multimedia/jpg...
3,3324753,01/12/2024,El sacrifici de l'equip de Dani Miret redueix ...,La Penya tomba el Madrid amb una exhibició def...,bàsquet,"[{'id': 'PUE3_BASQLLI', 'desc': 'Bàsquet - Lli...","[{'id': 'PENYA', 'desc': 'Penya', 'main': True}]",La Penya tomba el Madrid amb la millor defensa...,La Penya tomba el Madrid amb la millor defensa...,NaN,<!--RichText--><p>En el retorn d'Andrés Feliz ...,Redacció Esport3,[{'text': 'https://img.3cat.cat/multimedia/jpg...
4,3324728,01/12/2024,"Una nova llei, inèdita al món, reconeix que te...","Bèlgica, primer país a reconèixer permís de ma...",NaN,"[{'id': 'PUACT_MON', 'desc': 'Món', 'main': Tr...","[{'id': 'PROSTITUCIO', 'desc': 'Prostitució', ...","Bèlgica, primer país a reconèixer permís de ma...",NaN,NaN,<!--RichText--><p>Bèlgica s'ha convertit en el...,Redacció,[{'text': 'https://img.3cat.cat/multimedia/jpg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2388,3328932,31/12/2024,Hem resumit l'any amb una cinquantena de fotog...,El 2024 en un minut i deu segons: les 52 imatg...,NaN,"[{'id': 'PUACT_MON', 'desc': 'Món', 'main': Tr...",NaN,El 2024 en un minut i deu segons: les 52 imatg...,NaN,NaN,<!--RichText--><p>Un any són <strong>365 dies<...,Redacció,[{'text': 'https://img.3cat.cat/multimedia/jpg...
2389,3328917,31/12/2024,L'índex de referència baixa per novena vegada ...,"L'euríbor tanca l'any al voltant del 2,4% i to...",NaN,"[{'id': 'PUACT_ECONOM', 'desc': 'Economia (Act...","[{'id': 'ECOIFINANCES', 'desc': 'Economia', 'm...","L'euríbor tanca l'any al voltant del 2,4% i to...",NaN,NaN,<!--RichText--><p><strong>L'euríbor tancarà aq...,NaN,[{'text': 'https://img.3cat.cat/multimedia/jpg...
2390,3328826,31/12/2024,El 2024 acabarà amb domini del temps estable,"Dimarts, últim dia de l'any, sense canvis dest...",NaN,"[{'id': 'PUACT_PROMET', 'desc': 'Pronòstic met...","[{'id': 'METEOROLOGIA', 'desc': 'Meteorologia'...","Dimarts, últim dia de l'any, sense canvis dest...","Dimarts, últim dia de l'any, sense canvis dest...",NaN,<!--RichText--><p>L'<strong>anticicló</strong>...,Gori Masip,[{'text': 'https://img.3cat.cat/multimedia/jpg...
2391,3328944,31/12/2024,NaN,Un de cada tres doctorands de la UdG són d'ori...,NaN,NaN,NaN,Un de cada tres doctorands de la UdG són d'ori...,NaN,"[{'id': 'PUCOM_GIRONE', 'desc': 'Gironès', 'ma...",<!--RichText--><p>ACN Girona - Un terç dels do...,NaN,[{'text': 'https://img.3cat.cat/multimedia/jpg...


In [95]:
# Anem a extreure les noticies del 2024 del dataset df
df['data'] = pd.to_datetime(df['data'])

# Filtrar solo las filas donde el año sea 2024
noticies_2024 = df[df['data'].dt.year == 2024]

In [98]:
noticies_2024 = noticies_2024.sort_values(by='data', ascending=True)

In [103]:
# Abans d'ajuntar els dos df anem a posar el matex datetime format
# Convertir la columna "data" al formato de fecha
df_merged['data'] = pd.to_datetime(df_merged['data'], format='%d/%m/%Y')

# Cambiar el formato de la fecha a 'YYYY-MM-DD'
df_merged['data'] = df_merged['data'].dt.strftime('%Y-%m-%d')

In [104]:
# Anem a juntar els dos df
df_final = pd.concat([df_merged, noticies_2024], ignore_index=True)

df_final['data'] = pd.to_datetime(df_final['data'])
df_final = df_final.sort_values(by='data', ascending=True).reset_index(drop=True)

In [106]:
# Guardem el resultat en csv
df_final.to_csv('..//datasets//2024_news_dataset.csv', index=False)

In [2]:
df = pd.read_csv('..//datasets//2024_news_dataset.csv', low_memory=False)

In [5]:
# Anem a extreure les notícies del desembre
df["data"] = pd.to_datetime(df["data"])  # Convertir a tipo datetime si no lo está
df_desembre = df[df["data"].dt.month == 12]  # Filtrar solo diciembre

In [6]:
df_desembre.to_csv('..//datasets//2024_desembre_dataset.csv', index=False)

In [13]:
##############################################################################
# UNA VEGADA TENIM LES NOTÍCIES DEL 2024, ANEM A CREAR EL DATASET DE GUERRES #
##############################################################################

In [2]:
# Carreguem les notícies de desembre
df = pd.read_csv('..//datasets//2024_desembre_dataset.csv', low_memory=False)

In [3]:
# Carreguem la API KEY
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai_api_key

# Models disponibles
models = openai.models.list()
model_list = [model.id for model in models.data]
print(model_list)

['omni-moderation-2024-09-26', 'gpt-4o-mini-audio-preview-2024-12-17', 'dall-e-3', 'dall-e-2', 'gpt-4o-audio-preview-2024-10-01', 'gpt-4o-audio-preview', 'gpt-4o-mini-realtime-preview-2024-12-17', 'gpt-4o-mini-realtime-preview', 'o1-mini-2024-09-12', 'o1-preview-2024-09-12', 'o1-mini', 'o1-preview', 'gpt-4o-mini-audio-preview', 'whisper-1', 'gpt-4-turbo', 'gpt-4o-realtime-preview-2024-10-01', 'gpt-4', 'babbage-002', 'gpt-4-turbo-preview', 'tts-1-hd-1106', 'gpt-4-0125-preview', 'gpt-4o-audio-preview-2024-12-17', 'o3-mini-2025-01-31', 'o3-mini', 'tts-1-hd', 'gpt-4o-mini-2024-07-18', 'gpt-4o-2024-08-06', 'gpt-4o', 'tts-1', 'tts-1-1106', 'gpt-4-turbo-2024-04-09', 'davinci-002', 'gpt-3.5-turbo-1106', 'gpt-4o-mini', 'gpt-4o-2024-11-20', 'gpt-4o-2024-05-13', 'o1', 'gpt-3.5-turbo-instruct', 'o1-2024-12-17', 'chatgpt-4o-latest', 'gpt-3.5-turbo-instruct-0914', 'gpt-3.5-turbo-0125', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-3.5-turbo', 'gpt-4o-realtime-preview', 'gpt-3.5-turbo-16k', 'text-embedd

In [ ]:
template = '''
El següent text és el títol d'una notícia publicada a 3Cat, llegeix-lo i respon a les preguntes següents:

{titol}

Tasca: hauràs de determinar si la notícia pot tractar un tema de política Nacional (Espanya o Catalunya, no tinguis en compte notícies de política fora d'Espanya), si pot tractar sobre guerres o si tracta un tema diferent. 

Instrucció: respondràs amb un json amb el següent format:

{
    "tema": "tema de la notícia"
}

on "tema" pot ser "politica", "guerres" o "altres".
'''

In [17]:
client = OpenAI()


def generate_api_response_model_4o(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content

In [18]:
def generate_prompt(article, template):
    article_titol = article['titol']
    article_titol = str(article_titol)
    prompt = template.replace("{titol}", article_titol)
    return prompt

In [ ]:
# Quines son les tags més repetides?
import ast
from collections import Counter


# Quins tags es repeteixen mes?
def extract_ids(tag_str):
    try:
        tag_list = ast.literal_eval(tag_str)  
        return [tag['id'] for tag in tag_list]  
    except (ValueError, SyntaxError):
        return []  

all_ids = sum(df["tags"].apply(extract_ids).tolist(), [])

id_counts = Counter(all_ids)

id_counts_df = pd.DataFrame(id_counts.items(), columns=['id', 'count']).sort_values(by='count', ascending=False)

print(id_counts_df)

               id  count
40          SALUT    522
0        POLICIAL    471
59    ESTATSUNITS    447
21       JUDICIAL    395
9          ISRAEL    389
..            ...    ...
486   TELETREBALL      1
485  DONESCIENCIA      1
484           EGM      1
133      BB_SOPES      1
626  JUANMAMORENO      1

[627 rows x 2 columns]


In [40]:
# Anem a eliminar les noticies de meteorologia
import ast

def contains_meteorologia(tag_str):
    try:
        tag_list = ast.literal_eval(tag_str)  
        return any(tag['id'] == 'METEOROLOGIA' for tag in tag_list)  
    except (ValueError, SyntaxError):
        return False 

df2 = df2[~df2["tags"].apply(contains_meteorologia)]  

df2.reset_index(drop=True, inplace=True)



In [94]:
df2 = df.sample(50)


In [19]:
def repair_json(json_string):
    # Try to extract JSON (objects or arrays)
    json_match = re.search(r"(\{.*\}|\[.*\])", json_string, re.DOTALL)
    if json_match:
        json_content = json_match.group(0)
        try:
            parsed_json = json.loads(json_content)
            print("Valid JSON after extraction:", parsed_json)
            return parsed_json
        except json.JSONDecodeError as e:
            raise ValueError("No JSON object found in response")
    else:
        #throw error
        raise ValueError("No JSON object found in response")


In [20]:
results = {}
i = 0
length_df = len(df)

for _, row in df.iterrows():
    if i % 250 == 0:
        print(f"Processing row {i} of {length_df}")
    i +=1
    prompt = generate_prompt(row, template)
    response = generate_api_response_model_4o(prompt)
    id = row['id']

    # Check if the response is valid JSON
    try:
        response_json = json.loads(response)  # Try to parse the response as JSON
        results[row['id']] = response_json   # Store the parsed JSON under the id
    except json.JSONDecodeError:
        try:
            response_json = repair_json(response)
            results[row['id']] = response_json
        except ValueError as e:
            print("FATAL ERROR: Could not extract JSON from response")
            results[row['id']] = {"error": "Invalid JSON response"}
            break
    if i > 100:
        break

# Convert the results dictionary to a JSON string
final_json = json.dumps(results, indent=4)
print(final_json)

Processing row 0 of 2393
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'politica'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'guerres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'politica'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON

In [33]:
row = df.sample()

In [34]:
# print to a txt file the value in the column "cos"
with open('cos.txt', 'w') as f:
    f.write(row['cos'].values[0])

In [102]:
# Save to a file
with open('classificacio.json', 'w') as json_file:
    json_file.write(final_json)

print("JSON file has been saved as 'results.json'.")

JSON file has been saved as 'results.json'.


In [2]:
tags = pd.read_csv('..//datasets//tags.csv', low_memory=False)

In [28]:
# Define a list of keywords for filtering (Catalan terms for Artificial Intelligence)
keywords = ['intel·ligència artificial', 'aprenentatge automàtic', 'machine learning', 'Artificial intelligence', 'intel·ligència generativa']

# Combine relevant columns into a single text column for filtering
date_df['combined_text'] = (
    date_df['titol'].fillna('') + ' ' +
    date_df['entradeta'].fillna('') + ' ' +
    date_df['tags'].fillna('') + ' ' +
    date_df['tematiques'].fillna('')
)

# Filter rows containing any of the keywords
ai_news_df = date_df[date_df['combined_text'].str.contains('|'.join(keywords), case=False, na=False)]

# Drop the combined column if it's no longer needed
ai_news_df = ai_news_df.drop(columns=['combined_text'])


In [31]:
import ast

# Assuming your DataFrame is called `df`
# Extract the 'tags' column
tags_column = date_df['tags']

# Initialize a set to store unique IDs
unique_ids = set()

# Iterate through each row in the 'tags' column
for row in tags_column:
    if pd.notnull(row):  # Check for non-null values
        # Safely evaluate the string to a Python object
        parsed_row = ast.literal_eval(row)
        for item in parsed_row:
            unique_ids.add(item['id'])

# Convert the set to a sorted list (optional)
unique_ids = sorted(unique_ids)

# Display the unique IDs
print(unique_ids)


['ABALOS', 'ABASCAL', 'ABIDAL', 'ACATALANA', 'ACB', 'ACTUOCLIMA', 'ADACOLAU', 'AEROPORTPRAT', 'AEROPORTS', 'AFERNANDEZ', 'AFGANISTAN', 'AFRICA', 'AGRESSIOSEX', 'AGRICULTURA', 'AGTRIBUTARIA', 'AGUERO', 'AGULLO', 'AI', 'AIGUESBCN', 'AITANABONMAT', 'AJBARCELONA', 'ALBAVERGES', 'ALBERTRIVERA', 'ALBIACH', 'ALEMANYA', 'ALEXIAPUTELL', 'ALFREDBOSCH', 'ALGERIA', 'ALGUER', 'ALIMENTACIO', 'ALQAEDA', 'ALTSASU', 'ALVES', 'ALZHEIMER', 'AMARQUEZ', 'AMAZON', 'AMAZONIA', 'AMBLACULTURA', 'AMIANT', 'AMNISTIA', 'AN', 'ANC', 'ANDALUSIA', 'ANDORRA', 'ANGELSCHACON', 'ANIMAL', 'ANIVERS17A', 'ANNAERRA', 'ANNSIMO', 'ANOREXIA', 'ANSUFATI', 'ANTARTIDA', 'ANTIFEIXISME', 'ANY1983', 'ANY1984', 'ANY1986', 'ANY1987', 'ANY1988', 'ANY1989', 'ANY1990', 'ANY1991', 'ANY1992', 'ANY1993', 'ANY1994', 'ANY1995', 'ANY1996', 'ANY1997', 'ANY1998', 'ANY1999', 'ANY2000', 'ANY2001', 'ANY2002', 'ANY2004', 'ANY2005', 'ANY2006', 'ANY2007', 'ANY2008', 'ANY2009', 'ANY2010', 'ANY2011', 'ANY2012', 'ANY2013', 'ANY2014', 'ANY2015', 'ANY2016'

In [32]:
import ast

# Assuming your DataFrame is called `df`

# Filter rows where any dictionary in the 'tags' column contains an 'id' equal to "AI"
tag_ai = date_df[date_df['tags'].apply(
    lambda x: any(item['id'] == 'AI' for item in ast.literal_eval(x)) if pd.notnull(x) else False
)]

# Display the filtered DataFrame
print(tag_ai)


              data                                          entradeta  \
41802   2022-01-31  L'ONG de defensa dels drets humans Amnistia In...   
42173   2022-02-02  L'ONG fa públic un informe que acusa l'estat j...   
48599   2022-03-29  En l'informe anual, Amnistia Internacional den...   
60750   2022-07-18  Amnistia reclama suspendre la venda d'aquest t...   
70292   2022-10-25  Amnistia Internacional comunica la mort de més...   
72193   2022-11-10  Amnistia Internacional acusa Rússia de deporta...   
73511   2022-11-21  L'ONG considera que el Mundial pot servir per ...   
86153   2023-03-14  En els últims anys, a l'Estat hi ha hagut una ...   
87868   2023-03-28  Els "abusos que empara" la llei mordassa i la ...   
87891   2023-03-28  En l'informe anual, Amnistia constata un retro...   
112225  2023-11-11  Amnistia Internacional adverteix que la xarxa ...   
119879  2024-01-31  El president de l'entitat reclama reformar el ...   
128081  2024-04-24  L'ONG critica Espanya per la fa

In [34]:
import re

# List of keywords (the word 'dana' surrounded by word boundaries)
# keywords = ['\\bdana\\b', '\\briuades\\b', '\\btempestes\\b']  # Use \\b to ensure it's a whole word
keywords = ['\\bdana\\b']  # Use \\b to ensure it's a whole word

# Combine relevant columns into a single text column for filtering
date_df['combined_text'] = (
    date_df['titol'].fillna('') + ' ' +
    date_df['entradeta'].fillna('') + ' ' +
    date_df['tags'].fillna('') + ' ' +
    date_df['tematiques'].fillna('') + ' ' +
    date_df['permatitle'].fillna('') + ' ' +
    date_df['avantitol'].fillna('') + ' ' +
    date_df['titol_portada'].fillna('')
)

# Filter rows containing the exact word 'dana' (case-insensitive and word boundary)
dana_df = date_df[date_df['combined_text'].str.contains('|'.join(keywords), case=False, na=False, regex=True)]

# Drop the combined column if it's no longer needed
dana_df = dana_df.drop(columns=['combined_text'])

# Display the filtered DataFrame
print(dana_df)


              data                                          entradeta  \
26434   2021-09-07                                                NaN   
33991   2021-11-11  Un gran partit de la portera Montse Quesada i ...   
35188   2021-11-22  Arriba una DANA i es forma una pertorbació que...   
35386   2021-11-23  Protecció Civil i Bombers de la Generalitat fa...   
35542   2021-11-24  L'embossament d'aire fred o DANA se situa sobr...   
...            ...                                                ...   
150498  29/11/2024  "Joc de cartes", "Crims" o "La travessa", a de...   
150517  29/11/2024  El president valencià s'ha enfrontat amb minis...   
150518  29/11/2024  La Confederació Hidrogràfica del Xúquer va pre...   
150530  30/11/2024  Més de 100.000 persones han donat suport a la ...   
150567  30/11/2024  El concert "Barcelona amb València", amb Joan ...   

                                               permatitle avantitol  \
26434   La Ràpita demana la declaració de zona catas

In [38]:
import re

# List of keywords (the word 'dana' surrounded by word boundaries)
# keywords = ['\\bdana\\b', '\\briuades\\b', '\\btempestes\\b']  # Use \\b to ensure it's a whole word
keywords = ['\\bdana\\b']  # Use \\b to ensure it's a whole word

# Combine relevant columns into a single text column for filtering
id_df['combined_text'] = (
    id_df['cos'].fillna('') + ' ' +
    id_df['entradeta'].fillna('')
)

# Filter rows containing the exact word 'dana' (case-insensitive and word boundary)
dana_df_id = id_df[id_df['combined_text'].str.contains('|'.join(keywords), case=False, na=False, regex=True)]

# Drop the combined column if it's no longer needed
dana_df_id = dana_df_id.drop(columns=['combined_text'])

# Display the filtered DataFrame
print(dana_df_id)


             id                                          entradeta  \
1000    3070785                                                NaN   
4186    3076424                                                NaN   
5590    3078770  La fotògrafa Anne Lucy Silva ha guanyat el cer...   
6057    3079844  John Geddert, de 62 anys, s'ha suïcidat aquest...   
6387    3079948                                                NaN   
...         ...                                                ...   
147015  3324310  "Joc de cartes", "Crims" o "La travessa", a de...   
147034  3322793  El president valencià s'ha enfrontat amb minis...   
147035  3323970  La Confederació Hidrogràfica del Xúquer va pre...   
147047  3324650  Més de 100.000 persones han donat suport a la ...   
147084  3324593  El concert "Barcelona amb València", amb Joan ...   

                                                      cos           redactor  \
1000    ACN Vinaixa.-Els pagesos alerten que si els fr...                NaN   